Esta notebook es para ir a airbnb a chekear los precios de los alquileres, porque algunos no eran correctos.

In [2]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from datetime import datetime
import threading
lock = threading.RLock()
from os import walk

In [3]:
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])

proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
proxiesDf.head()


headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}

In [4]:
datasets_names = []
path_df = '../datasets/original_files/'
for (dirpath, dirnames, filenames) in walk(path_df):
    datasets_names.extend(filenames)
    break
dataFrame = False
first = True
for name in datasets_names:
    if(first):
        first = False
        dataFrame = pd.read_csv(path_df+name, compression='gzip')
        dataFrame.set_index('id', drop=False, inplace=True)
        print('adding', len(dataFrame), 'rows')
    else:
        _tmpDf = pd.read_csv(path_df+name, compression='gzip')
        _tmpDf.set_index('id', drop=False, inplace=True)
        print('adding', len(_tmpDf), 'rows')
        dataFrame = pd.concat([dataFrame, _tmpDf])
        del _tmpDf
dataFrame.loc[:,  'price'] = dataFrame.price.apply(lambda x: x[1:-3].replace(',', '')).astype(int)
print('total rows:', len(dataFrame))
dataFrame.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adding 11668 rows
adding 3799 rows
adding 4868 rows
adding 38481 rows
adding 7211 rows


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adding 50796 rows


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (43,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adding 13927 rows
total rows: 130750


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,
2265,2265,https://www.airbnb.com/rooms/2265,20200317143754,2020-03-17,Zen-East in the Heart of Austin (monthly rental),Zen East is situated in a vibrant & diverse mu...,This colorful and clean 1923 house was complet...,Zen East is situated in a vibrant & diverse mu...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,3,2,1,0,0.18
5245,5245,https://www.airbnb.com/rooms/5245,20200317143754,2020-03-17,"Eco friendly, Colorful, Clean, Cozy monthly share",Situated in a vibrant & diverse multicultural ...,"This green, colorful, clean and cozy house was...",Situated in a vibrant & diverse multicultural ...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,3,2,1,0,0.07
5456,5456,https://www.airbnb.com/rooms/5456,20200317143754,2020-03-17,"Walk to 6th, Rainey St and Convention Ctr",Great central location for walking to Convent...,Cute Private Studio apartment located in Willo...,Great central location for walking to Convent...,none,My neighborhood is ideally located if you want...,...,f,f,strict_14_with_grace_period,f,t,1,1,0,0,3.94
5769,5769,https://www.airbnb.com/rooms/5769,20200317143754,2020-03-17,NW Austin Room,NaN,Looking for a comfortable inexpensive room to ...,Looking for a comfortable inexpensive room to ...,none,Quiet neighborhood with lots of trees and good...,...,f,f,moderate,t,t,1,0,1,0,2.12
6413,6413,https://www.airbnb.com/rooms/6413,20200317143754,2020-03-17,Gem of a Studio near Downtown,"Great studio apartment, perfect for couples or...",!!!!! SXSW info !!!!! Presently open (again!) ...,"Great studio apartment, perfect for couples or...",none,Travis Heights is one of the oldest neighborho...,...,t,f,strict_14_with_grace_period,f,f,1,1,0,0,0.90


In [214]:
state = 'NY'
targets_df = dataFrame[dataFrame.state == state]
targets = targets_df.index

In [215]:
targets

Int64Index([20183694,     2060,     2595,     3831,     5099,     5114,
                5121,     5178,     5203,     5238,
            ...
            42890182, 42890276, 42890448, 42890474, 42890528, 42890680,
            42890730, 42891018, 42891637, 42892717],
           dtype='int64', name='id', length=50769)

In [216]:
def get_price(id):
    url = 'https://www.airbnb.com.ar/api/v2/pdp_listing_booking_details?_format=for_web_dateless&listing_id='+str(id)+'&currency=USD&guests=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&number_of_adults=1'
    response = requests.get(
        url,
        headers=headersChrome
    )
    print()
    data = json.loads(response.text)
    if(data['pdp_listing_booking_details'][0]['p3_display_rate']['currency'] == 'USD'):
        return int(data['pdp_listing_booking_details'][0]['p3_display_rate']['amount'])
    else:
        return None

In [220]:

get_price(20183694)

349

In [232]:
import pickle
real_prices = {}
import time

errors = 0
current = 0

for id in targets:
    current += 1
    if(errors == 25):
        time.sleep(60 * 60)
        errors = 0
    try:
        price = get_price(id)
        real_prices[id] = {
            'price': price,
            'time': time.time()
        }
        if current % 20 == 0:
            pickle.dump( real_prices, open( "../datasets/"+state+"_realPrices.pkl", "wb" ) )
        print('DONE', id)
        errors = 0
    except:
        errors += 1
        real_prices[id] = {
            'price': None,
            'time': time.time()
        }
        print('ERROR', id)
        
pickle.dump( real_prices, open( "../datasets/"+state+"_realPrices.pkl", "wb" ) )

DONE 20183694
DONE 2060
DONE 2595


In [233]:
pickle.load( open( "../datasets/"+state+"_realPrices.pkl", "rb" ) )

{20183694: {'price': 349, 'time': 1587957952.5370185},
 2060: {'price': 100, 'time': 1587957953.0513477},
 2595: {'price': 175, 'time': 1587957953.516776}}

In [224]:
real_prices

{20183694: 349, 2060: 100, 2595: 175}

In [50]:
dfFinal.loc[list(responses.keys()), 'real_price'] = list(responses.values())

In [53]:
# dfFinal[['real_price', 'price']][pd.notna(dfFinal.real_price)]

In [55]:
dfFinal.to_pickle('../datasets/final_dataset.pkl')